# Import & setup

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 100)


# Load dan gabungkan semua file query sintetis

In [2]:
base_path = "../data/processed/"

files = [
    "queries_part1.csv",
    "queries_part2.csv",
    "queries_part3.csv",
    "queries_part4.csv",
    "queries_part5.csv",
]

dfs = []
for f in files:
    path = base_path + f
    df_tmp = pd.read_csv(path)
    df_tmp["source_file"] = f  # untuk jejak asal
    dfs.append(df_tmp)

df_q = pd.concat(dfs, ignore_index=True)
df_q.head()


,seg_id,surah,ayah,tafsir_id,query_index,query_text,prompt_tokens,completion_tokens,total_tokens,source_file
0,1,Al-Fātiḥah,1,Al-Fātiḥah :1,1,Apa makna dari basmalah dalam memulai bacaan Al-Qur'an?,593,127,720,queries_part1.csv
1,1,Al-Fātiḥah,1,Al-Fātiḥah :1,2,Mengapa penting menyebut nama Allah sebelum melakukan pekerjaan?,593,127,720,queries_part1.csv
2,1,Al-Fātiḥah,1,Al-Fātiḥah :1,3,Bagaimana basmalah dapat memberikan keberkahan dalam aktivitas sehari-hari?,593,127,720,queries_part1.csv
3,1,Al-Fātiḥah,1,Al-Fātiḥah :1,4,Siapa yang dimaksud dengan Yang Maha Pengasih dan Maha Penyayang dalam tafsir ini?,593,127,720,queries_part1.csv
4,1,Al-Fātiḥah,1,Al-Fātiḥah :1,5,Apa perbedaan antara sifat kasih Allah kepada seluruh makhluk dan kepada orang beriman?,593,127,720,queries_part1.csv


# Lihat schema & missing values

In [3]:
df_q.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37557 entries, 0 to 37556
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   seg_id             37557 non-null  int64 
 1   surah              37557 non-null  object
 2   ayah               37557 non-null  int64 
 3   tafsir_id          37557 non-null  object
 4   query_index        37557 non-null  int64 
 5   query_text         37557 non-null  object
 6   prompt_tokens      37557 non-null  int64 
 7   completion_tokens  37557 non-null  int64 
 8   total_tokens       37557 non-null  int64 
 9   source_file        37557 non-null  object
dtypes: int64(6), object(4)
memory usage: 2.9+ MB


In [4]:
df_q.isna().sum().sort_values(ascending=False)


seg_id               0
surah                0
ayah                 0
tafsir_id            0
query_index          0
query_text           0
prompt_tokens        0
completion_tokens    0
total_tokens         0
source_file          0
dtype: int64

# Tetapkan nama kolom penting 
- kolom teks query = query_text
- kolom ID tafsir asal = tafsir_id_asal

In [ ]:
QUERY_COL = "query_text"      
TAFSIR_SRC_COL = "tafsir_id"  

df_q[[QUERY_COL, TAFSIR_SRC_COL]].head()


,query_text,tafsir_id
0,Apa makna dari basmalah dalam memulai bacaan Al-Qur'an?,Al-Fātiḥah :1
1,Mengapa penting menyebut nama Allah sebelum melakukan pekerjaan?,Al-Fātiḥah :1
2,Bagaimana basmalah dapat memberikan keberkahan dalam aktivitas sehari-hari?,Al-Fātiḥah :1
3,Siapa yang dimaksud dengan Yang Maha Pengasih dan Maha Penyayang dalam tafsir ini?,Al-Fātiḥah :1
4,Apa perbedaan antara sifat kasih Allah kepada seluruh makhluk dan kepada orang beriman?,Al-Fātiḥah :1


# Cek duplikasi & ukuran dataset

In [7]:
len(df_q), df_q[QUERY_COL].nunique()


(37557, 35764)

In [8]:
# duplikat full row
df_q.duplicated().sum()


np.int64(0)

In [9]:
# duplikat berdasarkan teks query saja
df_q[QUERY_COL].duplicated().sum()


np.int64(1793)

# Join ke tafsir_clean untuk cek mapping

In [10]:
df_tafsir = pd.read_csv("../data/processed/tafsir_clean.csv")

df_tafsir[["tafsir_id", "surah", "ayah"]].head()
df_tafsir.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   surah_id                6236 non-null   int64 
 1   surah                   6236 non-null   object
 2   ayah                    6236 non-null   int64 
 3   arabic_text             6236 non-null   object
 4   indonesian_translation  6236 non-null   object
 5   tafsir                  6236 non-null   object
 6   tafsir_id               6236 non-null   object
dtypes: int64(2), object(5)
memory usage: 341.2+ KB


In [11]:
df_join = df_q.merge(
    df_tafsir[["tafsir_id", "surah", "ayah"]],
    left_on=TAFSIR_SRC_COL,
    right_on="tafsir_id",
    how="left"
)

df_join.head()
df_join["tafsir_id"].isna().sum()


np.int64(0)

In [12]:
missing_mask = df_join["tafsir_id"].isna()
df_join[missing_mask].head(20)


,seg_id,surah_x,ayah_x,tafsir_id,query_index,query_text,prompt_tokens,completion_tokens,total_tokens,source_file,surah_y,ayah_y


# Tambah statistik panjang query

In [13]:
df_join["len_char"] = df_join[QUERY_COL].astype(str).str.len()
df_join["len_word"] = df_join[QUERY_COL].astype(str).str.split().str.len()

df_join[["len_char", "len_word"]].describe()


,len_char,len_word
count,37557.000000,37557.000000
mean,61.375988,8.637191
std,12.719436,1.841134
min,15.000000,2.000000
25%,53.000000,7.000000
50%,61.000000,9.000000
75%,69.000000,10.000000
max,187.000000,24.000000


# Lihat query yang terlalu pendek / terlalu panjang

In [15]:
# Query sangat pendek
df_join.sort_values("len_word").loc[:, [QUERY_COL, "len_word", "surah_x", "ayah_x"]].head(15)


,query_text,len_word,surah_x,ayah_x
13752,Siapa Zulkarnain?,2,Al-Kahf,92
13710,Siapa Zulkarnain?,2,Al-Kahf,85
16671,Mengapa mereka dibinasakan?,3,Al-Mu'minūn,48
16983,Apa itu barzakh?,3,Al-Mu'minūn,100
37249,Apa itu Hutamah?,3,Al-Humazah,6
37253,Kapan Hutamah dinyalakan?,3,Al-Humazah,6
35294,Apa itu sijjin?,3,Al-Muṭaffifīn,7
33331,Kapan Saqar terjadi?,3,Al-Muddaṡṡir,29
35374,Apa itu 'Illiyyin?,3,Al-Muṭaffifīn,20
18988,Siapa Nabi Syuaib?,3,Asy-Syu‘arā',178


In [17]:
# Query sangat panjang
df_join.sort_values("len_word", ascending=False).loc[:, [QUERY_COL, "len_word", "surah_x", "ayah_x"]].head(15)


,query_text,len_word,surah_x,ayah_x
20267,"Apa maksud dari pertanyaan 'Bagaimana pendapatmu, jika Allah menjadikan untukmu malam dengan kegelapan dan keheningannya itu terus-menerus demikian tanpa adanya siang sampai hari Kiamat?'",24,Al-Qaṣaṣ,71
22150,Apa maksud dari pernyataan bahwa Nabi Muhammad dan umatnya tidak akan dimintai tanggung jawab atas perbuatan orang lain di hari Kiamat?,21,Saba',25
16371,Bagaimana hubungan antara sifat Maha Mendengar dan Maha Melihat Allah dengan pengetahuan-Nya tentang masa lalu dan masa depan?,18,Al-Ḥajj,76
17444,Apa tiga waktu yang disebutkan dalam tafsir di mana hamba sahaya dan anak-anak harus meminta izin sebelum masuk?,18,An-Nūr,58
6886,Apa yang dimaksud dengan 'kebaikan selama hidup di dunia ini dan kelak di akhirat' dalam doa Nabi Musa?,18,Al-A‘rāf,156
10350,Apa maksud dari 'di atas setiap orang yang berpengetahuan pasti ada orang-orang yang lebih mengetahui' dalam konteks ini?,18,Yūsuf,76
28091,Apa maksud dari 'kitab yang berisi catatan tentang keadaan dan perbuatan mereka yang terpelihara baik' dalam tafsir ini?,18,Qāf,4
25944,Apa maksud dari 'segala apa yang ada di langit dan apa yang ada di bumi' dalam tafsir ini?,18,Asy-Syūrā,4
3922,Apa yang dimaksud dengan 'milik Allah-lah apa yang ada di langit dan apa yang ada di bumi'?,17,An-Nisā',132
24698,Apa perbedaan antara orang yang beribadah di malam hari dengan orang yang hanya berdoa saat tertimpa bencana?,17,Az-Zumar,9


# Distribusi jumlah query per surah (melalui join)

In [19]:
queries_per_surah = df_join.groupby("surah_x")["query_text"].count().reset_index(name="n_query")
queries_per_surah.sort_values("n_query", ascending=False).head(10)


,surah_x,n_query
9,Al-Baqarah,1820
77,Asy-Syu‘arā',1326
7,Al-A‘rāf,1255
110,Āli ‘Imrān,1227
67,An-Nisā',1110
91,Aṣ-Ṣāffāt,1050
3,Al-An‘ām,1022
82,At-Taubah,819
112,Ṭāhā,787
65,An-Naḥl,780


In [20]:
queries_per_surah["n_query"].describe()


count     114.000000
mean      329.447368
std       324.702453
min        16.000000
25%        95.000000
50%       232.500000
75%       478.000000
max      1820.000000
Name: n_query, dtype: float64

# Sampling sanity check query vs tafsir

In [22]:
sample = df_join.sample(10, random_state=42)

cols_show = ["surah_x", "ayah_x", QUERY_COL]
sample[cols_show]


,surah_x,ayah_x,query_text
31468,At-Tagābun,8,Siapa yang diajak untuk beriman dalam ayat ini?
14664,Ṭāhā,38,Apa yang direncanakan Fir'aun terhadap Nabi Musa?
35889,Al-A‘lā,5,Kapan rerumputan berubah menjadi kering dan kehitaman?
34318,An-Naba',18,Apa yang terjadi ketika sangkakala ditiup oleh Israfil?
32757,Nūḥ,5,Apa reaksi kaum Nabi Nuh terhadap ajakannya?
12307,An-Naḥl,94,Bagaimana cara bertobat dari berkhianat dalam sumpah?
35964,Al-A‘lā,18,Bagaimana cara membersihkan diri dari dosa menurut tafsir?
16887,Al-Mu'minūn,83,Siapa saja kaum terdahulu yang disebutkan memiliki perkataan serupa?
8318,At-Taubah,102,Apa yang dimaksud dengan orang yang mencampuradukkan pekerjaan baik dan buruk dalam konteks tobat?
8944,Yūnus,77,Apa yang ditanyakan Nabi Musa kepada Firaun dan pengikutnya?


In [24]:
sample_full = sample.merge(
    df_tafsir[["tafsir_id", "tafsir"]],
    left_on=TAFSIR_SRC_COL,
    right_on="tafsir_id",
    how="left"
)

sample_full[["surah_x", "ayah_x", QUERY_COL, "tafsir"]]


,surah_x,ayah_x,query_text,tafsir
0,At-Tagābun,8,Siapa yang diajak untuk beriman dalam ayat ini?,"Ayat ini mengajak orang-orang kafir untuk beriman dan mengajak orang-orang beriman yang ragu untuk benar-benar beriman. Maka berimanlah kamu, wahai seluruh manusia kepada Allah dan Rasul-Nya denga..."
1,Ṭāhā,38,Apa yang direncanakan Fir'aun terhadap Nabi Musa?,"Dan wahai Musa, ketahuilah bahwa sesungguhnya tanpa engkau minta pun, Kami telah memberi nikmat kepadamu pada kesempatan yang lain sebelum ini, yaitu ketika Kami mengilhamkan kepada ibumu sesudah ..."
2,Al-A‘lā,5,Kapan rerumputan berubah menjadi kering dan kehitaman?,"Rerumputan itu tumbuh, lalu setelah sekian lama dijadikan-Nya rerumputan itu kering dan berubah warna menjadi kehitam-hitaman. Begitulah siklus kehidupan di dunia: lahir, tumbuh, berkembang, matan..."
3,An-Naba',18,Apa yang terjadi ketika sangkakala ditiup oleh Israfil?,"Hari keputusan itu tiba pada hari ketika sangkakala ditiup oleh Israfil, lalu kamu akan bangkit dari kuburmu dan datang berbondong-bondong dan berduyun-duyun menuju tempat berkumpul, yaitu padang ..."
4,Nūḥ,5,Apa reaksi kaum Nabi Nuh terhadap ajakannya?,"Ajakan Nabi Nuh tidak disambut dengan baik oleh kaumnya, maka dia Nabi Nuh mengadu kepada Allah, “Ya Tuhanku, sesungguhnya aku telah menyeru kaumku untuk beriman kepada-Mu dengan berbagai cara, da..."
5,An-Naḥl,94,Bagaimana cara bertobat dari berkhianat dalam sumpah?,"Dan janganlah kamu berkhianat dengan men-jadikan sumpah-sumpahmu sebagai alat penipu di antaramu, yang menyebabkan kaki-mu tergelincir dan terjatuh setelah tegak dan mantap-nya di jalan yang benar..."
6,Al-A‘lā,18,Bagaimana cara membersihkan diri dari dosa menurut tafsir?,"Dasar-dasar ajaran agama samawi adalah sama, yaitu mereka yang beriman, beramal saleh, ingat kepada Allah, membersihkan diri dari dosa, dan memilih kehidupan akhirat akan berbahagia. Sebaliknya, m..."
7,Al-Mu'minūn,83,Siapa saja kaum terdahulu yang disebutkan memiliki perkataan serupa?,"Orang-orang kafir enggan memikirkan fenomena alam sebagai bukti kekuasaan Allah, bahkan mereka mengikuti jejak para pendurhaka terdahulu. Mereka mengingkari hari Kebangkitan dan mengucapkan perkat..."
8,At-Taubah,102,Apa yang dimaksud dengan orang yang mencampuradukkan pekerjaan baik dan buruk dalam konteks tobat?,"Dan ada pula orang lain yang berada di sekeliling kamu yang mengakui dosa-dosa mereka lalu bertobat atas dosa-dosa itu, tetapi mereka masih mencampuradukkan pekerjaan yang baik dengan pekerjaan la..."
9,Yūnus,77,Apa yang ditanyakan Nabi Musa kepada Firaun dan pengikutnya?,"Mendengar tuduhan Firaun dan pengikutnya, maka Nabi Musa berkata, “Pantaskah kamu mengatakan terhadap kebenaran ketika ia datang kepadamu, ‘sihirkah ini?’ Sungguh sangat tidak pantas. Padahal kamu..."


# Tambah kolom query_clean

In [27]:
output_path = "../data/processed/queries_synthetic_all.csv"

df_join.to_csv(output_path, index=False)
print("Saved to:", output_path)


Saved to: ../data/processed/queries_synthetic_all.csv


# Melihat total token untuk membuat query

In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/queries_synthetic_all.csv")
df.columns.tolist()


In [ ]:
total_prompt = int(df["prompt_tokens"].sum())
total_completion = int(df["completion_tokens"].sum())
total_total = int(df["total_tokens"].sum())

print("Total prompt tokens     :", total_prompt)
print("Total completion tokens :", total_completion)
print("Total tokens (input+out):", total_total)


Total prompt tokens     : 20597012
Total completion tokens : 4560769
Total tokens (input+out): 25157781


: 